In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import json
import ast
from tqdm import tqdm
import requests
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2

from core_function import *

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

## Links


In [17]:
links = pd.read_csv(os.path.join(pathfolder,'links.csv'))
links = links[links['tmdbId'].isnull()==False]
links.sort_values(by='tmdbId',inplace=True)
links['keyid'] = links['movieId'].astype('int').astype('str') + links['imdbId'].astype('int').astype('str') + links['tmdbId'].astype('int').astype('str')
links = links[['keyid','movieId','imdbId','tmdbId']]
links.sort_values(by='tmdbId',inplace=True)
# links = links.head(20000)

In [25]:
a1 = links.groupby('tmdbId').agg(idcount = ('tmdbId','count')).reset_index()
a1.sort_values(by='idcount',ascending=False,inplace=True)
a1 = a1[a1['idcount']> 1]

In [19]:
links_check = links[links['tmdbId'].isin(a1['tmdbId'])].sort_values(by='tmdbId')
links_check

,keyid,movieId,imdbId,tmdbId
34363,1446062702884912,144606,270288,4912.0
5892,60032905384912,6003,290538,4912.0
10103,343303680899775,34330,368089,9775.0
49997,1787553768009775,178755,376800,9775.0
48013,17453323567910991,174533,235679,10991.0
...,...,...,...,...
76772,25410014513892642732,254100,14513892,642732.0
68897,22158610394706703771,221586,10394706,703771.0
78451,26188312876132703771,261883,12876132,703771.0
74220,24210013323128792517,242100,13323128,792517.0


In [32]:
links['imdbId'].duplicated().sum()

0

In [33]:
for i in ['movieId','imdbId','tmdbId']:
    print(f'{i} duplicate = {links[i].duplicated().sum()}')

movieId duplicate = 0
imdbId duplicate = 0
tmdbId duplicate = 37


## ratings

In [22]:
movies_ratings = pd.read_csv(os.path.join(pathfolder,'ratings.csv'))
movies_ratings['keyid'] = movies_ratings['userId'].astype('str') + movies_ratings['movieId'].astype('str') + movies_ratings['timestamp'].astype('str')
movies_ratings['date_rate'] = pd.to_datetime(movies_ratings['timestamp'],unit='s')
del movies_ratings['timestamp']
movies_ratings.sort_values(by=['date_rate'],inplace=True)
movies_ratings = movies_ratings[['keyid','userId','movieId','rating','date_rate' ]]
# movies_ratings = movies_ratings.head(10000)
movies_ratings

,keyid,userId,movieId,rating,date_rate
15254009,1499541176789652004,149954,1176,4.0,1995-01-09 11:46:44
9617113,9453221789652009,94532,21,3.0,1995-01-09 11:46:49
9617119,9453247789652009,94532,47,5.0,1995-01-09 11:46:49
9617162,945321079789652009,94532,1079,3.0,1995-01-09 11:46:49
16958935,16647650822873600,166476,50,5.0,1996-01-29 00:00:00
...,...,...,...,...,...
22026732,2148312818641689841440,214831,281864,1.5,2023-07-20 08:24:00
29836322,29155452821689841759,291554,5282,3.5,2023-07-20 08:29:19
29836181,29155412011689842491,291554,1201,4.5,2023-07-20 08:41:31
29836625,2915542151731689842596,291554,215173,4.5,2023-07-20 08:43:16


In [31]:
len(set(movies_ratings[~movies_ratings['movieId'].isin(links['movieId'])]['movieId']))

123

In [26]:
len(set(movies_ratings[movies_ratings['movieId'].isin(links_check['movieId'])]['movieId']))

73

# Keywords

In [12]:
usecore = coreproc

q = '''
        select distinct movieid
        from staging.stg_links
        order by movieid 
    '''

stg_links = usecore.selectdf(q)
stg_links_list = list(set(stg_links['movieid'].values))
stg_links_list.sort()

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

moives_tags = pd.read_csv(os.path.join(pathfolder,'tags.csv'))
moives_tags['date_tag'] = pd.to_datetime(moives_tags['timestamp'],unit='s')
moives_tags['tag'] = moives_tags['tag'].str.lower().str.strip()
moives_tags['tag'] = moives_tags['tag'].replace('',np.nan)
moives_tags.rename(columns={'tag':'keywords'},inplace=True)
moives_tags['keyid'] = moives_tags['userId'].astype('int').astype('str') + moives_tags['movieId'].astype('int').astype('str') + moives_tags['timestamp'].astype('int').astype('str')
moives_tags = moives_tags[['keyid','userId','movieId','keywords','date_tag']]
moives_tags = moives_tags[moives_tags['movieId'].isin(stg_links_list)]
moives_tags

,keyid,userId,movieId,keywords,date_tag
0,102601430666558,10,260,good vs evil,2015-05-03 15:22:38
1,102601430666505,10,260,harrison ford,2015-05-03 15:21:45
2,102601430666538,10,260,sci-fi,2015-05-03 15:22:18
3,1412211311600756,14,1221,al pacino,2011-07-25 13:32:36
4,1412211311600746,14,1221,mafia,2011-07-25 13:32:26
...,...,...,...,...,...
2328198,330923855101507019604,330923,85510,women as sexual objects,2017-10-03 08:33:24
2328311,33093333171351279384,330933,3317,coming of age,2012-10-26 19:23:04
2328312,33093333171351279389,330933,3317,sexuality,2012-10-26 19:23:09
2328313,33094757821154110902,330947,5782,not luc besson,2006-07-28 18:21:42


In [11]:
list(set(stg_links['movieid'].values))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 131076,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 131104,
 34,
 131106,
 36,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 131116,
 52,
 131118,
 54,
 131126,
 131124,
 57,
 58,
 131128,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 131138,
 74,
 75,
 76,
 131142,
 78,
 79,
 131144,
 81,
 82,
 131154,
 131150,
 85,
 86,
 87,
 88,
 89,
 131158,
 131156,
 92,
 93,
 94,
 95,
 96,
 97,
 100,
 101,
 103,
 104,
 105,
 106,
 107,
 262251,
 110,
 111,
 112,
 117,
 118,
 121,
 122,
 123,
 124,
 125,
 126,
 131200,
 130,
 132,
 135,
 137,
 138,
 140,
 141,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 131237,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 131248,
 180,
 181,
 131250,
 131252,
 184,
 185,
 186,
 187,
 188,
 131254,
 190,
 19

## Metadata

In [11]:
sql_check = '''
            select distinct *
            from staging.stg_tmdb_json_movie_metadata stjmm 
            where request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
            order by id
            '''
movie_json = coreuat.selectdf(sql_check)
# list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
movie_json.sort_values(by='id',inplace=True)

data_dict_list = []

for i in tqdm(set(movie_json['id'])):
    data_str = movie_json[movie_json['id']==i]['request_json'].values[0]
    data = f'''{data_str}'''
    data_dict = json.loads(data)
    data_dict_list.append(data_dict)
    
movie_metadata = pd.DataFrame(data_dict_list)
movie_metadata.sort_values(by='id',inplace=True)
movie_metadata


100%|██████████| 85304/85304 [00:26<00:00, 3213.97it/s]


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,2,tt0094675,[FI],fi,...,1988-10-21,0,73,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,None,Ariel,False,7.129,345
2,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,3,tt0092149,[FI],fi,...,1986-10-17,0,74,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,None,Shadows in Paradise,False,7.306,407
3,False,/f2t4JbUvQIjUF5FstG1zZFAp02N.jpg,None,4000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",https://www.miramax.com/movie/four-rooms/,5,tt0113101,[US],en,...,1995-12-09,4257354,98,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.900,2683
4,False,/iUUpKunmBN5l8goObADBaFHnxQ8.jpg,None,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",None,6,tt0107286,[US],en,...,1993-10-15,12136938,109,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.500,346
5,False,/2w4xG178RpB4MDAIfTkqAuSJzec.jpg,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,11,tt0076759,[US],en,...,1977-05-25,775398007,121,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.203,21056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49780,False,None,None,0,[],None,1144338,None,[],en,...,None,0,0,[],Canceled,None,Table,False,0.000,0
50074,False,/4OIJmgez12DITjHbWiNf1C7uG1d.jpg,None,0,"[{'id': 37, 'name': 'Western'}]",None,1145358,tt23469212,[US],en,...,2023-07-07,0,96,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Play your cards right and you just might survive.,Dead Man's Hand,False,5.778,9
50369,False,/t86SkVd35W9COtCGtbxxb3UoQpW.jpg,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",None,1146246,tt6810618,[CA],en,...,2024-08-22,0,84,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,None,The Deep Web: Murdershow,False,4.600,5
50371,False,/eNopYAzvtfvaj1yCI0AeVAOlHzU.jpg,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",None,1146249,tt17519798,[US],en,...,2023-07-12,0,88,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Insane in the membrane.,Cabin Girl,False,5.100,19


In [12]:
movie_metadata.columns.to_list()

['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'origin_country',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [13]:
for i in ['id','imdb_id']:
    print(f'{i} duplicate = {movie_metadata[i].duplicated().sum()}')

id duplicate = 0
imdb_id duplicate = 19


In [19]:
movie_metadata2 = movie_metadata[['id','title','original_title','original_language','release_date','status','overview','tagline','adult','popularity','homepage','poster_path','runtime','budget','revenue','vote_average','vote_count']]
movie_metadata2.head(5)

,id,title,original_title,original_language,release_date,status,overview,tagline,adult,popularity,homepage,poster_path,runtime,budget,revenue,vote_average,vote_count
1,2,Ariel,Ariel,fi,1988-10-21,Released,A Finnish man goes to the city to find a job a...,None,False,1.5038,None,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,73,0,0,7.129,345
2,3,Shadows in Paradise,Varjoja paratiisissa,fi,1986-10-17,Released,"Nikander, a rubbish collector and would-be ent...",None,False,1.1210,None,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,74,0,0,7.306,407
3,5,Four Rooms,Four Rooms,en,1995-12-09,Released,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,False,3.1509,https://www.miramax.com/movie/four-rooms/,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,98,4000000,4257354,5.900,2683
4,6,Judgment Night,Judgment Night,en,1993-10-15,Released,"Four young friends, while taking a shortcut en...",Don't move. Don't whisper. Don't even breathe.,False,1.1798,None,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,109,21000000,12136938,6.500,346
5,11,Star Wars,Star Wars,en,1977-05-25,Released,Princess Leia is captured and held hostage by ...,"A long time ago in a galaxy far, far away...",False,19.8534,http://www.starwars.com/films/star-wars-episod...,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,121,11000000,775398007,8.203,21056


## Creadit

In [2]:
q = '''
    with 
        metadata as
            (
                select distinct *
                from staging.stg_tmdb_json_movie_metadata stjmm 
                where request_json = '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
                order by id		
            )
    select distinct *
    from staging.stg_tmdb_json_movie_credits stjmc 
    where id not in (select distinct id from metadata)
        and request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
    order by id 
    '''
creadit = coreuat.selectdf(q)
creadit

,id,request_json
0,2,"{""id"": 2, ""cast"": [{""adult"": false, ""gender"": ..."
1,3,"{""id"": 3, ""cast"": [{""adult"": false, ""gender"": ..."
2,5,"{""id"": 5, ""cast"": [{""adult"": false, ""gender"": ..."
3,6,"{""id"": 6, ""cast"": [{""adult"": false, ""gender"": ..."
4,11,"{""id"": 11, ""cast"": [{""adult"": false, ""gender"":..."
...,...,...
37048,100313,"{""id"": 100313, ""cast"": [{""adult"": false, ""gend..."
37049,100318,"{""id"": 100318, ""cast"": [{""adult"": false, ""gend..."
37050,100323,"{""id"": 100323, ""cast"": [{""adult"": false, ""gend..."
37051,100327,"{""id"": 100327, ""cast"": [{""adult"": false, ""gend..."


In [173]:
cast_crew_list = []

for i in tqdm((set(creadit['id']))):
    datastr = creadit[creadit['id']==i]['request_json'].values[0]
    data = json.loads(f'''{datastr}''')
    cast_crew_list.append(data)
   
creaditdf = pd.DataFrame(cast_crew_list)
creaditdf.sort_values(by='id',inplace=True)
creaditdf

100%|██████████| 37053/37053 [00:21<00:00, 1764.19it/s]


,id,cast,crew
0,2,"[{'adult': False, 'gender': 2, 'id': 54768, 'k...","[{'adult': False, 'gender': 2, 'id': 16767, 'k..."
1,3,"[{'adult': False, 'gender': 2, 'id': 4826, 'kn...","[{'adult': False, 'gender': 1, 'id': 54766, 'k..."
2,5,"[{'adult': False, 'gender': 2, 'id': 3129, 'kn...","[{'adult': False, 'gender': 0, 'id': 3116, 'kn..."
3,6,"[{'adult': False, 'gender': 2, 'id': 2880, 'kn...","[{'adult': False, 'gender': 2, 'id': 2042, 'kn..."
4,11,"[{'adult': False, 'gender': 2, 'id': 2, 'known...","[{'adult': False, 'gender': 2, 'id': 1, 'known..."
...,...,...,...
37048,100313,"[{'adult': False, 'gender': 2, 'id': 31771, 'k...","[{'adult': False, 'gender': 2, 'id': 930633, '..."
37049,100318,"[{'adult': False, 'gender': 2, 'id': 33923, 'k...","[{'adult': False, 'gender': 2, 'id': 551674, '..."
37050,100323,"[{'adult': False, 'gender': 1, 'id': 121038, '...","[{'adult': False, 'gender': 2, 'id': 138172, '..."
37051,100327,"[{'adult': False, 'gender': 2, 'id': 31771, 'k...","[{'adult': False, 'gender': 2, 'id': 551674, '..."


In [174]:
# id = tmdbid movie
cast_raw = creaditdf[['id','cast']]

castdf_list = []

listrun = creaditdf['id']

# listrun = [2]

for i in tqdm(set(listrun)):
    dataf = cast_raw[cast_raw['id']==i]['cast'].values[0]
    numb_cast = len(dataf)
    for y in range(len(dataf)):
        cast_id = dataf[y]['id']
        dataf[y]['movieid'] = i
        if dataf[y]['profile_path'] is not None:
            dataf[y]['profile_path'] = 'https://image.tmdb.org/t/p/w500' + dataf[y]['profile_path']
        castdf_list.append(dataf[y])

stg_cast = pd.DataFrame(castdf_list)
stg_cast = stg_cast[stg_cast.known_for_department == 'Acting']
print(f'''credit_id' duplicate = {stg_cast['credit_id'].duplicated().sum()}''')
stg_cast = stg_cast[['credit_id', 'movieid','id', 'cast_id', 'character']]
stg_cast.rename(columns={'movieid':'id','id':'actor_id'},inplace=True)
stg_cast

  0%|          | 0/37053 [00:00<?, ?it/s]

100%|██████████| 37053/37053 [00:09<00:00, 3920.58it/s]


credit_id' duplicate = 0


,credit_id,id,actor_id,cast_id,character
0,52fe420dc3a36847f8000029,2,54768,3,Taisto Kasurinen
1,52fe420dc3a36847f800002d,2,54769,4,Irmeli Pihlaja
2,52fe420dc3a36847f8000031,2,4826,5,Mikkonen
3,52fe420dc3a36847f8000035,2,54770,6,Riku
4,5e1cda97d5dbc20017f439bb,2,1177496,12,Miner
...,...,...,...,...,...
870264,52fe49d4c3a36847f81a88ad,100329,1169622,1007,Senén
870265,52fe49d4c3a36847f81a88b1,100329,967362,1008,Felisa
870267,52fe49d4c3a36847f81a88b9,100329,1023096,1010,Doctor
870268,52fe49d4c3a36847f81a88bd,100329,1169623,1011,Cura


In [ ]:
stg_cast[stg_cast.credit_id == '59b5a875c3a3686890001111']

,credit_id,id,actor_id,cast_id,character
538344,59b5a875c3a3686890001111,43211,1885525,23,"Paolo Fabbris, detto ""Paolone"""


In [177]:
stg_cast[stg_cast.credit_id == '5c2527260e0a265deb342136']

,credit_id,id,actor_id,cast_id,character
670648,5c2527260e0a265deb342136,59105,2202911,18,None


In [171]:
ctest = stg_cast[stg_cast.index==538344]
ctest

,credit_id,crew_id,actor_id,cast_id,character
538344,59b5a875c3a3686890001111,43211,1885525,23,"Paolo Fabbris, detto ""Paolone"""


In [176]:
stg_cast[stg_cast.actor_id == 1885525]

,credit_id,id,actor_id,cast_id,character
538344,59b5a875c3a3686890001111,43211,1885525,23,"Paolo Fabbris, detto ""Paolone"""


In [169]:
sql_table_name = 'cast'
python_table = ctest
sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"
sql

'INSERT INTO cast VALUES (%s,%s,%s,%s,%s)'

In [170]:
data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]
data

Preparing data: 100%|██████████| 1/1 [00:00<00:00, 382.06it/s]


[('59b5a875c3a3686890001111',
  43211,
  1885525,
  23,
  'Paolo Fabbris, detto "Paolone"')]

In [164]:
# id = tmdbid movie
crew_raw = creaditdf[['id','crew']]

crewdf_list = []

listrun = creaditdf['id']

# listrun = [2]

for i in tqdm(set(listrun)):
    dataf = crew_raw[crew_raw['id']==i]['crew'].values[0]
    numb_crew = len(dataf)
    for y in range(len(dataf)):
        crew_id = dataf[y]['id']
        dataf[y]['movieid'] = i
        if dataf[y]['profile_path'] is not None:
            dataf[y]['profile_path'] = 'https://image.tmdb.org/t/p/w500' + dataf[y]['profile_path']
        crewdf_list.append(dataf[y])

stg_crew = pd.DataFrame(crewdf_list)
stg_crew = stg_crew[stg_crew.known_for_department != 'Acting']
print(f'''credit_id' duplicate = {stg_crew['credit_id'].duplicated().sum()}''')
stg_crew = stg_crew[['credit_id', 'movieid','id', 'department', 'job']]
stg_crew.rename(columns={'movieid':'id','id':'crew_id'},inplace=True)
stg_crew

  0%|          | 0/37053 [00:00<?, ?it/s]

100%|██████████| 37053/37053 [00:08<00:00, 4310.15it/s]


credit_id' duplicate = 0


,credit_id,id,crew_id,department,job
0,52fe420dc3a36847f8000025,2,16767,Directing,Director
1,52fe420dc3a36847f800003b,2,16769,Camera,Director of Photography
2,52fe420dc3a36847f8000041,2,54766,Editing,Editor
3,52fe420dc3a36847f8000047,2,53836,Art,Production Design
4,52fe420dc3a36847f800004d,2,54771,Costume & Make-Up,Costume Design
...,...,...,...,...,...
1035818,52fe49d4c3a36847f81a8895,100329,1024245,Writing,Writer
1035819,52fe49d4c3a36847f81a88d3,100329,6525,Writing,Co-Writer
1035820,52fe49d4c3a36847f81a88cd,100329,15604,Editing,Editor
1035821,52fe49d4c3a36847f81a88c7,100329,63975,Camera,Director of Photography


In [120]:
for i in ['credit_id']:
    print(f'{i} duplicate = {stg_crew[i].duplicated().sum()}')

credit_id duplicate = 0
